<a href="https://colab.research.google.com/github/mfernl/03MIAR_Algoritmos-de-optimizacion-2026/blob/main/Trabajo_practico_Tribunales_TFM_mfernl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Marco Fernández Llamas  <br>
Url: https://github.com/mfernl/03MIAR_Algoritmos-de-optimizacion-2026.git<br>
Google Colab: https://colab.research.google.com/drive/10vcf6mztWS27BvXNkXRYOVHwaeoioQCq?usp=sharing<br>
Problema:
Configuración de Tribunales

Descripción del problema:
Se precisa configurar tribunales de evaluación para un grupo de 15 alumnos que desean presentar su Trabajo Fin de Máster (TFM).
Cada tribunal está compuesto por tres profesores, cada uno desempeñando uno de los siguientes roles: Presidente, Secretario o Vocal.
Los profesores han indicado su disponibilidad horaria para participar en los tribunales de 15h a 21h durante la semana del 15 al 19 de abril.

Hay 15 alumnos, por lo que se deben configurar 15 tribunales buscando la configuración más equilibrada posible. Además ningún profesor puede ser convocado a dos tribunales al mismo tiempo ni a un tribunal al que no tenga disponibilidad.






                                        

#Implementación de la solución

In [7]:
import io
import pandas as pd
import numpy as np

disponibilidad = pd.read_csv(io.StringIO('''
0,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0
1,1,1,1,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
0,0,1,1,0,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,1,0,1,0,1
1,0,1,0,1,1,0,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0
1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,0
1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1,1,0,1,0,1
0,1,1,1,1,1,1,1,1,0,1,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,1,1,1,1,1,1,0,1
1,1,1,1,1,0,0,1,1,0,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0
1,0,1,1,0,1,0,0,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1
1,1,0,1,1,0,1,1,0,0,0,0,0,1,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1
'''), names=['15-15','15-16','15-17','15-18','15-19','15-20','15-21','16-15','16-16','16-17','16-18','16-19','16-20','16-21','17-15','17-16','17-17','17-18','17-19','17-20','17-21','18-15','18-16','18-17','18-18','18-19','18-20','18-21','19-15','19-16','19-17','19-18','19-19','19-20','19-21',], )
disponibilidad.index = ['RRD','QYV','LHL','HLC','MSB','PMQ','QWF','EBB','IOE','IOA']

roles = pd.read_csv(io.StringIO('''
Profesor,ROL
RRD,"P,S,V"
QYV,"P,S,V"
LHL,"P,V"
HLC,"S,V"
MSB,"P,S,V"
PMQ,"P,S,V"
QWF,"S,V"
EBB,"S,V"
IOE,"P,S,V"
IOA,"P,S,V"
'''),header = 0,index_col=0)

sol = pd.DataFrame(data = np.zeros((15,4)),columns = ['Fecha-Hora', 'Presidente', 'Secretario', 'Vocal'], dtype='string')
roles['num_Trib'] = np.zeros(len(roles))

In [8]:
import itertools
import pandas as pd

def resolver_tribunales(disponibilidad, roles):

    profesores = list(disponibilidad.index)
    franjas = list(disponibilidad.columns)

    carga = {p: 0 for p in profesores}
    solucion = []

    roles_dict = {p: roles.loc[p]['ROL'].split(',') for p in profesores}

    # Generar combinaciones válidas en una franja
    def combinaciones_validas(franja):

        disponibles = sorted([p for p in profesores if disponibilidad.loc[p, franja] == 1],key=lambda x: carga[x]) #ordenar por carga, evita sobrecargar profesores
        validas = []

        for p, s, v in itertools.permutations(disponibles, 3):
            if carga[p] >= 5 or carga[s] >= 5 or carga[v] >= 5:
                continue

            if ('P' in roles_dict[p] and 'S' in roles_dict[s] and 'V' in roles_dict[v]):
              validas.append((p, s, v))

        return validas


    def backtrack(franja_idx, tribunales_asignados):

        if tribunales_asignados == 15: #Solución encontrada
            return True

        if franja_idx == len(franjas): #No se haya solución
            return False

        franjas_restantes = len(franjas) - franja_idx
        tribunales_restantes = 15 - tribunales_asignados

        if franjas_restantes < tribunales_restantes: #No quedan franjas para poner tribunales
            return False

        franja = franjas[franja_idx]

        if backtrack(franja_idx + 1, tribunales_asignados): #Probar con la siguiente franja
            return True

        for p, s, v in combinaciones_validas(franja):

            carga[p] += 1
            carga[s] += 1
            carga[v] += 1

            solucion.append({
                'Fecha-Hora': franja,
                'Presidente': p,
                'Secretario': s,
                'Vocal': v
            })

            if backtrack(franja_idx + 1, tribunales_asignados + 1): #Solución válida
                return True

            carga[p] -= 1
            carga[s] -= 1
            carga[v] -= 1
            solucion.pop() #poda

        return False


    if backtrack(0, 0):
        display(carga) #Se observa la carga de cada profesor
        return pd.DataFrame(solucion)
    else:
        return None


In [9]:
s = resolver_tribunales(disponibilidad,roles)
display(s)

{'RRD': 5,
 'QYV': 5,
 'LHL': 5,
 'HLC': 5,
 'MSB': 4,
 'PMQ': 5,
 'QWF': 4,
 'EBB': 4,
 'IOE': 4,
 'IOA': 4}

,Fecha-Hora,Presidente,Secretario,Vocal
0,17-21,RRD,HLC,LHL
1,18-15,QYV,PMQ,QWF
2,18-16,MSB,EBB,IOE
3,18-17,RRD,QYV,LHL
4,18-18,IOA,HLC,MSB
5,18-19,IOE,EBB,IOA
6,18-20,PMQ,QWF,RRD
7,18-21,QYV,HLC,LHL
8,19-15,MSB,PMQ,QWF
9,19-16,IOA,EBB,RRD


#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

Para trabajar en el problema se ha utilizado la librería **pandas**, la cual permite trabajar con un tipo de datos multidimensional denominado DataFrame. Un tipo de datos basado en filas y columnas, en las que se almacenan las disponibilidades de horario de los profesores y los roles que pueden ejercer cada profesor.

Para la solución, se usa un dataframe (sol) en el que se almacenan los 15 tribunales como filas. Como columnas, el presidente, el secretario, el vocal y la franja horaria para cada tribunal.

La función objetivo del algoritmo es repartir 15 tribunales entre 10 profesores de la manera más equilibrada posible. Cada tribunal está formado por 3 roles, esto quiere decir que 3 roles por 15 tribunales se tienen 45 huecos que rellenar por profesores, entre 10 profesores, tocan a 4.5 tribunales por profesor que se redondean a 5 tribunales unos y 4 tribunales otros.

Para hayar esta solución, se imponen como restricciones que la carga de los profesores no puede ser mayor de 5 tribunales. Además, se busca siempre que se comprueben primero para cada franja horaria, los profesores que menos tribunales tienen asignados. De esta manera, se asegura que los huecos son repartidos de manera equilibrada.

In [10]:
#Aquí se muestran los dataframes de disponibilidades, roles, y donde se almacena la solución
display(disponibilidad)
display(roles)
display(sol)

,15-15,15-16,15-17,15-18,15-19,15-20,15-21,16-15,16-16,16-17,...,18-19,18-20,18-21,19-15,19-16,19-17,19-18,19-19,19-20,19-21
RRD,0,1,1,1,0,1,1,1,0,1,...,1,1,1,1,1,1,1,1,0,0
QYV,1,1,1,1,0,0,0,0,1,1,...,1,1,1,1,1,1,1,1,1,1
LHL,0,0,1,1,0,1,1,1,1,1,...,1,0,1,0,1,1,0,1,0,1
HLC,1,0,1,0,1,1,0,1,0,0,...,0,1,1,1,1,1,1,1,1,0
MSB,1,1,0,1,0,1,1,1,1,1,...,0,1,1,1,0,1,1,1,1,0
PMQ,1,1,1,1,1,0,0,1,1,1,...,0,1,1,1,1,1,0,1,0,1
QWF,0,1,1,1,1,1,1,1,1,0,...,0,1,1,1,1,1,1,1,0,1
EBB,1,1,1,1,1,0,0,1,1,0,...,1,1,1,0,1,1,1,0,1,0
IOE,1,0,1,1,0,1,0,0,1,1,...,1,1,0,1,0,1,1,1,1,1
IOA,1,1,0,1,1,0,1,1,0,0,...,1,1,1,1,1,1,0,0,0,1


,ROL,num_Trib
Profesor,,
RRD,"P,S,V",0.0
QYV,"P,S,V",0.0
LHL,"P,V",0.0
HLC,"S,V",0.0
MSB,"P,S,V",0.0
PMQ,"P,S,V",0.0
QWF,"S,V",0.0
EBB,"S,V",0.0
IOE,"P,S,V",0.0


,Fecha-Hora,Presidente,Secretario,Vocal
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

El algoritmo de backtracking realiza una búsqueda entre todas las posibles soluciones al problema, aunque se descarten soluciones no válidas, se está generando un árbol de búsqueda en el que se escogen 15 franjas horarias de 35 en total, esto da un total de $\binom{35}{15}$ posibles soluciones.

La complejidad del algoritmo al ser de backtracking será exponencial en el peor de los casos. Dependiendo del número de combinaciones de profesores disponibles en cada franja ($C$), la complejidad es exponencial sobre el número de tribunales que se quieren crear, O($C^{15}$).

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

A la hora de abordar el problema, se probó con un algoritmo voraz que asignara a tribunales a los profesores que estuvieran libres, pero las características del problema impedían llegar a la solución óptima. Es por esto que finalmente se ha escogido un algoritmo de **backtracking**, el cual permite elegir la solución óptima de entre todas las soluciones sacrificando tiempo y espacio. Para esta cantidad de datos, resulta imperceptible la diferencia de complejidad con el algoritmo voraz.